# Recommender Systems
This notebook contains the first part of assignment 1 about recommender systems.

By:  
Antoni Czernek (S4000595) (a.a.czernek@umail.leidenuniv.nl)  
Art Schenkel (S3745244) (j.a.schenkel@umail.leidenuniv.nl)  
Sadaf Esmaeili Rad (S3986160) (sadafismaeili@gmail.com)  

## Loading dependencies and reading in the data
The first cell of this notebook is used for importing the needed dependencies. The following cells read in the data from the MovieLens 1M dataset. We will mainly use the rating_df for learning and testing.

In [2]:
# Importing dependencies
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
movies_df = pd.read_csv('ml-1m/movies.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['Movie_Id', 'movie_name', 'genre'], encoding = "ISO-8859-1")

In [4]:
rating_df = pd.read_csv('ml-1m/ratings.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['user_id', 'Movie_Id','Ratings','Time_stamp'], encoding = "ISO-8859-1")

In [5]:
users_df = pd.read_csv('ml-1m/users.dat',
                        delimiter='::', header=None,
                        names=['user_id', 'Gender','Age','Occupation','Zip-Code'], encoding = "ISO-8859-1")

## Cross-validation
The first part of the code implements the cross-validation. The cross-validation funtion return a dataframe with a randomly assigned number of folds. This dataframe is then used to learn and test the models, by choosing one of the fold as a valid test, and the other as the train sets.

In [6]:
def cross_validation(df,n_folds):
    shuffled_df = df.sample(random_state = 42, frac =1)
    shuffled_df['Fold']= None
    shuffled_df.reset_index(inplace = True)
    shuffled_df.drop(columns = 'index', inplace = True)
    data_size = len(shuffled_df)
    for i in range(1,n_folds):
        shuffled_df.loc[int((i-1)/n_folds*data_size):int(i/n_folds * data_size),'Fold'] = i
    shuffled_df.loc[int((n_folds-1)/n_folds*data_size):,'Fold']= n_folds
    return shuffled_df

In [7]:
data = cross_validation(rating_df, 5)
data.reset_index(inplace = True)

## Naive Approaches
We will now start with the naive approaches. These are: global average (ratingGlobal()), movie average (ratingItem()), user average (ratingUser()) and a linear combination of the three averages (ratingUserItem() and ratingUserStarItem). We also calculate the optimal parameters for linear regression by applying the least-squares solution algorithm.

In [8]:
# rating global, return mean of all ratings
def ratingGlobal():
    return rating_df["Ratings"].mean()

print("Global rating:" , ratingGlobal())

Global rating: 3.581564453029317


In [9]:
# rating item, return mean of all ratings for a specific item
def ratingItem(item):
    join = pd.merge(movies_df, rating_df, how='left', on="Movie_Id")
    result = join[join["Movie_Id"] == item]
    return result["Ratings"].mean()

print("Item rating:", ratingItem(1193))

Item rating: 4.390724637681159


In [10]:
# rating user, return mean of all rating for a specific user
def ratingUser(user):
    join = pd.merge(users_df, rating_df, how='left', on="user_id")
    result = join[join["user_id"] == user]
    return result["Ratings"].mean()

print("User Rating:", ratingUser(3))

User Rating: 3.9019607843137254


In [11]:
# This helper function returns a rating given a user_id and movie_id
def rating(user, item):
    result1 = rating_df[rating_df["user_id"] == user]
    result2 = result1[result1["Movie_Id"] == item]
    return int(result2.Ratings)

print(rating(1, 661))

3


In [12]:
# rating user item, combines user rating and item rating multiplied by paramer alpha and beta respectively. Lastly parameter gamma is added
def ratingUserItem(user, item, alpha, beta, gamma):
    result = alpha * ratingUser(user) + beta * ratingItem(item) + gamma

    # make sure the result is a valid rating, between 1 and 5
    if(result > 5): result = 5
    if(result < 1): result = 1

    return result

print("User Item Rating:", ratingUserItem(1, 1193, 0.41, 0.34, 0.09))
print("actual rating:", rating(1, 661))

User Item Rating: 3.300204867377632
actual rating: 3


In [13]:
# rating user star item is similar to rating user item, but wwithout adding parameter gamma at the end
def ratingUserStarItem(user, item, alpha, beta):
    result = alpha * ratingUser(user) + beta * ratingItem(item)

    # make sure the result is a valid rating, between 1 and 5
    if(result > 5): result = 5
    if(result < 1): result = 1
    
    return result

print("User* Item Rating:", ratingUserStarItem(1, 661, 0.5, 0.5))

User* Item Rating: 3.826720575022462


In [14]:
# This helper function returns a rating given a user_id and movie_id
def rating(user, item):
    result1 = rating_df[rating_df["user_id"] == user]
    result2 = result1[result1["Movie_Id"] == item]
    return int(result2.Ratings)

print(rating(1, 661))

3


In [40]:
# This function calculates the optimal value for parameters alpha, beta and gamma for a specific user and movie using linear regression.
def linearRegression(user, item):
    avguser = np.array([ratingUser(user)])
    avgmovie = np.array([ratingItem(item)])
    currrating = np.array([rating(user, item)])

    a = np.column_stack((avguser, avgmovie, np.ones_like(avguser)))
    b = currrating

    x, residuals, rank, s = np.linalg.lstsq(a, b, rcond=None)

    alpha, beta, gamma = x

    print("Optimal values:")
    print("Alpha:", alpha)
    print("Beta:", beta)
    print("Gamma:", gamma)

linearRegression(1,661)

Optimal values:
Alpha: 0.4113321969726794
Beta: 0.34024284095705803
Gamma: 0.09820092990789195


In the following function we calculate the root mean squared error and mean absolute error for the five naive approaches. We only test them on the first 100 ratings (takes ca. 90 second) in the dataset. This is because testing on all one million datapoints would take approximately 250 hours with our current implementation. We are not exactly sure what causes these particular functions to be so slow, but one reason might be that some of them require a merge on the dataset for each of the iterations. 

In [19]:
def testNaive():
    testRatings = []

    predGlobal = []
    predItems = []
    predUsers = []
    predUsersItems = []
    predUsersStarItem = []

    data2 = data.head(100)
    for i in range(len(data2)):
        # save user, item and rating
        user = data2.loc[i]['user_id']
        item = data2.loc[i]['Movie_Id']
        rating = data2.loc[i]['Ratings']
        testRatings.append(rating)
        # append the lists with predictions
        predGlobal.append(ratingGlobal())
        predItems.append(ratingItem(item))
        predUsers.append(ratingUser(user))
        predUsersItems.append(ratingUserItem(user, item, 0.5, 0.5, 0.1))
        predUsersStarItem.append(ratingUserStarItem(user, item, 0.5, 0.5))
    # print the mean squared error and mean absolute error for each approach
    # rating global
    print("RMSE rating global: ", mean_squared_error(testRatings, predGlobal, squared=False))
    print("MAE rating global: ", mean_absolute_error(testRatings, predGlobal))
    # rating item
    print("RMSE rating item: ", mean_squared_error(testRatings, predItems, squared=False))
    print("MAE rating item: ", mean_absolute_error(testRatings, predItems))
    # rating user
    print("RMSE rating user: ", mean_squared_error(testRatings, predUsers, squared=False))
    print("MAE rating user: ", mean_absolute_error(testRatings, predUsers))
    # rating user item
    print("RMSE rating user item: ", mean_squared_error(testRatings, predUsersItems, squared=False))
    print("MAE rating user item: ", mean_absolute_error(testRatings, predUsersItems))
    # rating user star item
    print("RMSE rating user star item: ", mean_squared_error(testRatings, predUsersStarItem, squared=False))
    print("MAE rating user star item: ", mean_absolute_error(testRatings, predUsersStarItem))

testNaive()

RMSE rating global:  1.218032202986966
MAE rating global:  1.0385809765758955
RMSE rating item:  1.1197185168344852
MAE rating item:  0.913625132487781
RMSE rating user:  1.0599107303804303
MAE rating user:  0.8793486849986037
RMSE rating user item:  1.0581103789204882
MAE rating user item:  0.8668951353322589
RMSE rating user star item:  1.0494506983688492
MAE rating user star item:  0.876932847453439


## UV Matrix Decomposition
Next, we implemented UV matrix decomposition as described in section 9.4 of the MMDS textbook. 

As in the Matrix Factorization algorithm implemented next, we use dictionaries to translate the movie_id and user_id to matrix_id

In this algorithm in case of a movie_id (or user_id) appearing in the test example, but not present in the train set we simply use the average of the whole set as the prediction. A different approach is used in the Matrix Factorization part.

In [45]:
def uvMatrixDecomp():
    # Load the rating data
    DR = rating_df

    # Set hyperparameters and the Kfold implemented in sklearn
    kf = KFold(n_splits = 5 , shuffle = True, random_state = 42)
    c = 2 
    i = 5 # Number of iterations

    test_results_rmse =[]
    test_results_mae =[]
    # Split the data into training and testing sets
    for train_index , test_index in kf.split(DR):
        DR_train, DR_test = DR.loc[train_index], DR.loc[test_index]

        Row = DR_train.pivot(index = 'user_id', columns ='Movie_Id', values = 'Ratings')

        #Creating a dictionaries for the 
        values_user = np.unique(DR_train['user_id'])
        user_dict = {values_user[idxx] : idxx for idxx in range(len(values_user))}
        values_movie = np.unique(DR_train['Movie_Id'])
        movie_dict = {values_movie[idxx] : idxx for idxx in range(len(values_movie))}

        # Calculating the means and creating the matrix that we will then learn on
        # that is, the matrix of user_id and movie_id as rows and columns
        u_mean = Row.mean(axis=1)
        Row_array = Row.to_numpy()
        u_mean = u_mean.to_numpy()

        normal = Row_array # - u_mean.reshape(-1,1)
        N = normal

        #initializing the values of the U and V vectors, as simply just ones
        u = np.full((normal.shape[0],2), 1)
        v = np.full((2,normal.shape[1]), 1)
        u = u.astype(np.float32)
        v = v.astype(np.float32)

        #calculating the UV matrix
        uv = np.dot(u,v)

        print("TRAIN:", train_index, "TEST:", test_index)

        # Perform matrix factorization
        for iterations in range(i):
            # Updating the rows of the U matrix
            for r in range(Row_array.shape[0]):

                for s in range(c):
                    sums = 0
                    u_rk = u[r,:]
                    v_kj = v[:,:]
                    
                    # This part below is used to calculate the Prj element, as it uses the sum
                    # over U and V without the s row
                    u_rk_del = np.delete(u_rk, s, 0)
                    v_kj_del = np.delete(v_kj, s, 0)
                    v_sj = v[s,:]
                    v_sj_squared = v_sj ** 2

                    u_rk_v_kj = np.dot(u_rk_del, v_kj_del)
                    m_rj = N[r,:]

                    error = m_rj - u_rk_v_kj

                    # calculating the new value for the U[r,s]
                    vsj_dot_er = v_sj * error
                    sums = np.nansum(vsj_dot_er)
                    v_sj_ssum = np.nansum((v_sj_squared) * (~np.isnan(m_rj)))
                    newval_u = sums / v_sj_ssum
                    u[r,s] = u[r,s] + ((newval_u - u[r,s]))
            
            # Below is the update part for the V matrix, which is very similar to the U update
            for r in range(c):
                for s in range(Row_array.shape[1]):
                    sums = 0
                
                    u_ik = u[:,:]
                    v_ks = v[:,s]
                    u_ik_del = np.delete(u_ik, r, 1)

                    v_ks_del = np.delete(v_ks, r, 0)
                    u_ir = u[:,r]
                    u_ir_squared = u_ir ** 2

                    u_ik_v_ks = np.dot(u_ik_del, v_ks_del)
                    m_is = N[:,s]
                    error = m_is - u_ik_v_ks

                    uir_dot_er = u_ir * error
                    sumsv = np.nansum(uir_dot_er)
                    u_ir_ssum = np.nansum(u_ir_squared * (~np.isnan(m_is)))
                    newval_v = sumsv / u_ir_ssum
                    v[r,s] = v[r,s] + ((newval_v - v[r,s]))

            # Calculate and show the Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE)
            uv = np.dot(u,v)
            dif = uv -normal

            print("Iteration Number: ",iterations )

            dif_abs= (np.absolute(dif))
            dif_abs_0s = np.nan_to_num(dif_abs)
            dif_abs_sum = np.sum(dif_abs_0s,axis=0)
            sum_dif = dif_abs_sum.sum()
            non_0_count = np.count_nonzero(dif_abs_0s)
            MAE=sum_dif/non_0_count

            # print('MAE',MAE)

            dif_sqr = dif ** 2
            dif_sqr_0s = np.nan_to_num(dif_sqr)
            dif_sqr_total= np.sum( dif_sqr_0s ,axis=0)
            sumz = dif_sqr_total.sum()
            non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
            RME = sumz/ non_0_count_sqr
            rme_list=[RME]

            # print('RMSE=',RME)

        # Testing part, we get the prediction matrix
        pred_matrix = np.dot(u,v)
        preds = []

        #We calculate the number of failed examples - examples where either movie_id or user_id
        # does not appear in the train set but appears in the test_set
        fails = 0
        for idx in range(len(DR_test)):
            try:
                prediction = pred_matrix[user_dict[DR_test.iloc[idx]['user_id']]][movie_dict[DR_test.iloc[idx]['Movie_Id']]]
            except:
                # For the mismatching of the user_id or movie_id we just use the global average rating
                fails += 1
                prediction = 3.58
            preds.append(prediction)
        labels = DR_test['Ratings']
        print(f'% failure: {fails/len(DR_test)}')
        test_rmse =  mean_squared_error(labels,np.array(preds), squared = False)
        test_mae = mean_absolute_error(labels,np.array(preds))
        test_results_rmse.append(test_rmse)
        test_results_mae.append(test_mae)
        print(test_rmse)
    print('Mean results from K-Fold:')
    print(f'RMSE: {np.mean(np.array(test_results_rmse))} \nMAE: {np.mean(np.array(test_results_mae))}')


In [46]:
# perform the UV matrix decomposition
uvMatrixDecomp()

TRAIN: [      1       2       3 ... 1000205 1000207 1000208] TEST: [      0       7      12 ... 1000196 1000198 1000206]
Iteration Number:  0
Iteration Number:  1
Iteration Number:  2
Iteration Number:  3
Iteration Number:  4
% failure: 0.00012997270573179633
0.9141455882399945
TRAIN: [      0       1       2 ... 1000205 1000206 1000208] TEST: [      6       9      10 ... 1000192 1000200 1000207]
Iteration Number:  0
Iteration Number:  1
Iteration Number:  2
Iteration Number:  3
Iteration Number:  4
% failure: 0.00016996430749542597
0.9090654506647099
TRAIN: [      0       1       3 ... 1000205 1000206 1000207] TEST: [      2      11      26 ... 1000201 1000204 1000208]
Iteration Number:  0
Iteration Number:  1
Iteration Number:  2
Iteration Number:  3
Iteration Number:  4
% failure: 0.00014996850661361114
0.9082958701869176
TRAIN: [      0       2       3 ... 1000206 1000207 1000208] TEST: [      1       4      16 ... 1000175 1000193 1000199]
Iteration Number:  0
Iteration Number:  1


## Matrix Factorization
Lastly we implemented matrix factorization as described in the gravity-Tikk paper. We create the model as two matrixes of features for the users and movies. Sizes of the matrixes are determined by the unique number of user_id (accordingly movie_id for the item matrix) and the number of features that is chosen when creating the model. 

As the train set may include gaps in the numbering of either user_id or movie_id we use dictionaries to translate the movie_id (or user_id) to matrix_id

For the learning part, we use the algorithm described on the slides from the lecture, we calculate the prediction as a scalar multiplication of the according vectors in matrixes of users and items. Then we calculate the error and update the vectors by learning_rate*(error*user_matrix_vector- lamb*self.item_matrix_vector). As we can see there are two parameters learning rate and lambda that we can tune.

We initialize the matrixes with random numbers using np.random.rand function

The output is put into [1:5] interval by simply setting every output < 1 into 1 and every output > 5 as 5, other outputs are not changed.

In the testing function, when the test example uses user_id that did not appear in the train set (in case of a movie_id not appearing in the train set proceed similarly) we take the sum of the vectors for the corresponding movie_id and scale the sums of features of all movie_ids into a [1:5] interval, and read the value of the movie_id that is asked in the testing example (after the scaling). 

The assumption is that the bigger the sum of features the bigger the chance is for the movie to get a high score. This assumption is not 100% true, as for example having feature < 0 with a corresponding <0 feature in the user_id vector creates a positive result.

In [12]:
class MatrixFactorization:
    def __init__(self,x, num_features):
        #initilaze two matrixes that then multiply by each other to give a matrix of ratings
        
        user_size = np.unique(x['user_id']).shape[0]
        item_size = np.unique(x['Movie_Id']).shape[0]

        values_user = np.unique(x['user_id'])
        self.dict_user = {values_user[i] : i for i in range(len(values_user))}

        values_item = np.unique(x['Movie_Id'])
        self.dict_item = {values_item[i] : i for i in range(len(values_item))}
        
        self.user_matrix = np.random.rand(user_size,num_features)
        self.item_matrix = np.random.rand(item_size,num_features)
        
    def fit(self,x, learning_rate = 0.005, lamb = 0.05, n_iter = 10):
        for it in range(n_iter):
            tmp = 0
            for i in range(len(x)):
                user = x.loc[i]['user_id']
                item = x.loc[i]['Movie_Id']

                user_idx = self.dict_user[user]
                item_idx = self.dict_item[item]
                
                #calculate the error
                error = x.loc[i]['Ratings'] - min(max(np.matmul(self.user_matrix[user_idx],self.item_matrix[item_idx]),1),5)
                # update values
                self.user_matrix[user_idx] = self.user_matrix[user_idx] + learning_rate*(error*self.item_matrix[item_idx] - lamb*self.user_matrix[user_idx])
                self.item_matrix[item_idx] = self.item_matrix[item_idx] + learning_rate*(error*self.user_matrix[user_idx] - lamb*self.item_matrix[item_idx])

                tmp += 1
                if tmp%50000 ==0:
                    print(f'currently done: {tmp/len(x)} % of the iteration {it}')

        
        print('current iteration ended: '+str(it))
    def test(self,x):
        predictions = []
        for i in range(len(x)):
            
            user = x.loc[i]['user_id']
            item = x.loc[i]['Movie_Id']

            try:
                user_idx = self.dict_user[user]
                item_idx = self.dict_item[item]
                pred = min(max(np.matmul(self.user_matrix[user_idx],self.item_matrix[item_idx]),1),5)
                predictions.append(pred)
                
            except: #If there is no user
                try:
                    item_idx = self.dict_item[item]
                    sum_item = np.sum(self.item_matrix[item_idx])
                    sums = np.sum(self.item_matrix, axis = 1)
                    pred =  (sum_item- np.min(sums)) / (np.max(sums) - np.min(sums)) * (4) + 1
                    predictions.append(pred)
                
                except: # If there is no movie
                    user_idx = self.dict_user[user]
                    sum_user= np.sum(self.user_matrix[item_idx])
                    sums = np.sum(self.user_matrix, axis = 1)
                    pred =  (sum_user - np.min(sums)) / (np.max(sums) - np.min(sums)) * (4) + 1
                    predictions.append(pred)
                #calculate the error
            
        labels = np.array(x['Ratings'])
        predictions = np.array(predictions)
        rmse =  mean_squared_error(labels,predictions, squared = False)
        mse = mean_absolute_error(labels,predictions)

        return rmse, mse

Here are some of the results we managed to get using this method with different parameters. The best results we got from the parameters: 

n_features = 10
n_iter = 15
learning_rate = 0.01
lambda = 0.03 

comment - We have noticed an error in the implementation of the MatrixFactorization that the number of unique movie_id was taken from the dataset itself, so the matrix had all of them implemented (at random of course). The learning on these sets underneath is on the wrongly implemented constructor. We believe that this error would not change the results by a lot. Unfortunately, as the function fit time takes hours, especially with that number of iterations, and the error was found on the day of the deadline we couldn't refit the functions. We provide the old implementation under the training. 

As we have seen before, the number of cases when the movie_id or user_id cannot be found in the matrix is around 0.01%, and as the initialization of that vector is random the changes in the final results should be very low, or even marginal

In [17]:
rmse_list = []
mae_list = []
for i in range(1,6):
    train = data.loc[data['Fold'] != i ]
    # print(train)
    train = train.reset_index()
    valid = data.loc[data['Fold'] == i ] 
    valid = valid.reset_index()
    mt = MatrixFactorization(train,50)
    # print(train)
    print('Fitting the fold: ' + str(i))
    mt.fit(train, n_iter = 10)
    rmse, mae = mt.test(valid)
    print(f"Fold {i} RMSE: {rmse} \n MSE: {mae}")
    rmse_list.append(rmse)
    mae_list.append(mae)
print(f'Mean results:\nRMSE: {np.mean(np.array(rmse_list))} \nMAE: {np.mean(np.array(mae_list))}')

Fitting the fold: 1
currently done: 0.062486877755671306 % of the iteration 0
currently done: 0.12497375551134261 % of the iteration 0
currently done: 0.18746063326701393 % of the iteration 0
currently done: 0.24994751102268523 % of the iteration 0
currently done: 0.31243438877835655 % of the iteration 0
currently done: 0.37492126653402785 % of the iteration 0
currently done: 0.43740814428969915 % of the iteration 0
currently done: 0.49989502204537045 % of the iteration 0
currently done: 0.5623818998010418 % of the iteration 0
currently done: 0.6248687775567131 % of the iteration 0
currently done: 0.6873556553123844 % of the iteration 0
currently done: 0.7498425330680557 % of the iteration 0
currently done: 0.812329410823727 % of the iteration 0
currently done: 0.8748162885793983 % of the iteration 0
currently done: 0.9373031663350696 % of the iteration 0
currently done: 0.9997900440907409 % of the iteration 0
currently done: 0.062486877755671306 % of the iteration 1
currently done: 0.

In [12]:
rmse_list = []
mae_list = []
for i in range(1,6):
    train = data.loc[data['Fold'] != i ]
    train = train.reset_index()
    valid = data.loc[data['Fold'] == i ] 
    valid = valid.reset_index()
    mt_2 = MatrixFactorization(train,20)
    print('Fitting the fold: ' + str(i))
    mt_2.fit(train,learning_rate = 0.01, lamb = 0.03, n_iter = 15)
    rmse, mae = mt_2.test(valid)
    print(f"Fold {i} RMSE: {rmse} \n MSE: {mae}")
    rmse_list.append(rmse)
    mae_list.append(mae)
print(f'Mean results:\nRMSE: {np.mean(np.array(rmse_list))} \nMAE: {np.mean(np.array(mae_list))}')

Fitting the fold: 1
currently done: 0.062486877755671306 % of the iteration 0
currently done: 0.12497375551134261 % of the iteration 0
currently done: 0.18746063326701393 % of the iteration 0
currently done: 0.24994751102268523 % of the iteration 0
currently done: 0.31243438877835655 % of the iteration 0
currently done: 0.37492126653402785 % of the iteration 0
currently done: 0.43740814428969915 % of the iteration 0
currently done: 0.49989502204537045 % of the iteration 0
currently done: 0.5623818998010418 % of the iteration 0
currently done: 0.6248687775567131 % of the iteration 0
currently done: 0.6873556553123844 % of the iteration 0
currently done: 0.7498425330680557 % of the iteration 0
currently done: 0.812329410823727 % of the iteration 0
currently done: 0.8748162885793983 % of the iteration 0
currently done: 0.9373031663350696 % of the iteration 0
currently done: 0.9997900440907409 % of the iteration 0
currently done: 0.062486877755671306 % of the iteration 1
currently done: 0.

In [13]:
c_rmse = 10000
user_params = None
item_params = None
rmse_list = []
mae_list = []
for i in range(1,6):
    train = data.loc[data['Fold'] != i ]
    # print(train)
    train = train.reset_index()
    valid = data.loc[data['Fold'] == i ] 
    valid = valid.reset_index()
    mt_3 = MatrixFactorization(train,20)
    # print(train)
    print('Fitting the fold: ' + str(i))
    mt_3.fit(train,learning_rate = 0.002, lamb = 0.04, n_iter = 10)
    rmse, mae = mt_3.test(valid)
    print(f"Fold {i} RMSE: {rmse} \n MSE: {mae}")
    rmse_list.append(rmse)
    mae_list.append(mae)
print(f'Mean results:\nRMSE: {np.mean(np.array(rmse_list))} \nMAE: {np.mean(np.array(mae_list))}')
    # if rmse < c_rmse:
    #     user_params = mt.user_matrix
    #     item_params = mt.item_matrix

Fitting the fold: 1
currently done: 0.062486877755671306 % of the iteration 0
currently done: 0.12497375551134261 % of the iteration 0
currently done: 0.18746063326701393 % of the iteration 0
currently done: 0.24994751102268523 % of the iteration 0
currently done: 0.31243438877835655 % of the iteration 0
currently done: 0.37492126653402785 % of the iteration 0
currently done: 0.43740814428969915 % of the iteration 0
currently done: 0.49989502204537045 % of the iteration 0
currently done: 0.5623818998010418 % of the iteration 0
currently done: 0.6248687775567131 % of the iteration 0
currently done: 0.6873556553123844 % of the iteration 0
currently done: 0.7498425330680557 % of the iteration 0
currently done: 0.812329410823727 % of the iteration 0
currently done: 0.8748162885793983 % of the iteration 0
currently done: 0.9373031663350696 % of the iteration 0
currently done: 0.9997900440907409 % of the iteration 0
currently done: 0.062486877755671306 % of the iteration 1
currently done: 0.

### Savin the best model for the visualization part
We save the best performing model by saving the matrixes and the dictionaries it used

In [13]:
np.save('user_matrix.npy',mt_2.user_matrix)
np.save('item_matrix.npy',mt_2.item_matrix)

In [24]:
mt_2.dict_user
list_user = list(mt_2.dict_user.items())
np.save('dict_user.npy',np.array(list_user))

In [22]:
mt_2.dict_item
key_list_item = list(mt_2.dict_item.keys())
item_list_item = list(mt_2.dict_item.items())
np.save('dict_items.npy',np.array(item_list_item))
# np.save(np.array(key))

## Conclusion part one
Based on the testing results, the Matrix Factorization performed the best achieving the mean RMSE score of ~0.888 which is a ~6.6% improvement over the Netflix base treshhold. Although this algorithm gives good results it takes a lot of time to fit, a few hours with only 15 interactions, which makes it difficult to find the best hyperparameters (learning_rate and lambda).

The second best was the UV matrix decomposition with a mean RMSE score of ~ 0.910 which also is a good improvement. The algorithm was much faster in the terms of the fitting time.

This concludes the first part of our report about recommender systems. For the data visualisation of the results of our matrix factorization method, see part two of the report.

## Apendix 1 - Wrongly implemented MatrixFactorization

Here is the old version of the MatrixFactorization, the only change is in the item_size implementation

In [ ]:
class MatrixFactorization:
    def __init__(self,x, num_features):
        #initilaze two matrixes that then multiply by each other to give a matrix of ratings
        
        user_size = np.unique(x['user_id']).shape[0]

        #Here is the error
        item_size = np.unique(data['Movie_Id']).shape[0]

        values_user = np.unique(x['user_id'])
        self.dict_user = {values_user[i] : i for i in range(len(values_user))}

        # key_list_user = list(dict_user.keys())

        values_item = np.unique(x['Movie_Id'])
        self.dict_item = {values_item[i] : i for i in range(len(values_item))}
        # key_list_item = list(dict_item.keys())

        
        self.user_matrix = np.random.rand(user_size,num_features)
        self.item_matrix = np.random.rand(item_size,num_features)
        
    def fit(self,x, learning_rate = 0.005, lamb = 0.05, n_iter = 10):
        for it in range(n_iter):
            tmp = 0
            for i in range(len(x)):
                user = x.loc[i]['user_id']
                item = x.loc[i]['Movie_Id']

                user_idx = self.dict_user[user]
                item_idx = self.dict_item[item]
                
                #calculate the error
                error = x.loc[i]['Ratings'] - min(max(np.matmul(self.user_matrix[user_idx],self.item_matrix[item_idx]),1),5)
                # update values
                self.user_matrix[user_idx] = self.user_matrix[user_idx] + learning_rate*(error*self.item_matrix[item_idx] - lamb*self.user_matrix[user_idx])
                self.item_matrix[item_idx] = self.item_matrix[item_idx] + learning_rate*(error*self.user_matrix[user_idx] - lamb*self.item_matrix[item_idx])

                tmp += 1
                if tmp%50000 ==0:
                    print(f'currently done: {tmp/len(x)} % of the iteration {it}')

        
        print('current iteration ended: '+str(it))
    def test(self,x):
        predictions = []
        for i in range(len(x)):
            
            user = x.loc[i]['user_id']
            item = x.loc[i]['Movie_Id']

            try:
                user_idx = self.dict_user[user]
                item_idx = self.dict_item[item]
                pred = min(max(np.matmul(self.user_matrix[user_idx],self.item_matrix[item_idx]),1),5)
                predictions.append(pred)
                
            except: #If there is no user
                try:
                    item_idx = self.dict_item[item]
                    sum_item = np.sum(self.item_matrix[item_idx])
                    sums = np.sum(self.item_matrix, axis = 1)
                    pred =  (sum_item- np.min(sums)) / (np.max(sums) - np.min(sums)) * (4) + 1
                    predictions.append(pred)
                
                except: # If there is no movie
                    user_idx = self.dict_user[user]
                    sum_user= np.sum(self.user_matrix[item_idx])
                    sums = np.sum(self.user_matrix, axis = 1)
                    pred =  (sum_user - np.min(sums)) / (np.max(sums) - np.min(sums)) * (4) + 1
                    predictions.append(pred)
                #calculate the error
            
        labels = np.array(x['Ratings'])
        predictions = np.array(predictions)
        rmse =  mean_squared_error(labels,predictions, squared = False)
        mse = mean_absolute_error(labels,predictions)

        return rmse, mse